In [1]:
from openai import OpenAI
client = OpenAI()

In [ ]:
image_url = "https://images.pickles.com.au/image/upload/f_auto,q_auto,w_2000/c_scale,g_south_east,l_pickles-logo-white_mipoap.png,o_60,w_0.4,fl_relative,x_0.02,y_0.02,fl_region_relative/v1726789702/asostfbejshw7ekod5xa.jpg"
image_url

In [ ]:
response = client.chat.completions.create(
  model="gpt-4o-mini",
  messages=[
    {
      "role": "user",
      "content": [
        {
            "type": "text", 
            "text": "What's in this image, answer in one senetence?"
        },
        {
          "type": "image_url",
          "image_url": {
            "url": image_url
          },
        }
      ]
    }
  ],
  max_tokens=100
)

print(response.choices[0])

In [13]:
image_path = "car_damage/bumper_dent--minor--0001.jpeg"

In [ ]:
import base64

def encode_image(image_path):
  with open(image_path, "rb") as image_file:
    return base64.b64encode(image_file.read()).decode("utf-8")

base64_image = encode_image(image_path)
image_url  = f"data:image/jpeg;base64,{base64_image}"

response = client.chat.completions.create(
  model="gpt-4o-mini",
  messages=[
    {
      "role": "user",
      "content": [
        {
          "type": "text", 
          "text": "What's in this image, answer in one senetence?"
        },
        {
          "type": "image_url",
          "image_url": {
            "url": image_url
          }
        }
      ]
    }
  ],
  max_tokens=100
)

print(response.choices[0])

In [ ]:
{
    "messages": [
        {
            "role": "system", 
            "content": "You are an assistant that identifies which part of the car is damaged and the severity level of the damage"
        },
        {
            "role": "user", 
            "content": "Identify which part of the car is damaged and the severity level of the damage."
        },
        {
            "role": "user", 
            "content": [
                {
                    "type": "image_url", 
                    "image_url": { 
                        "url": image_url
                    }
                }
            ]
        },
        {
            "role": "assistant", 
            "content": "doors"
        }
    ]
}

In [95]:
from pydantic import BaseModel

class damage_output(BaseModel):
    damaged_part: str
    damge_type: str
    damage_severity: str

In [96]:
import glob
import json

with open("car_damage_train.jsonl", "w") as fp:
    for image_path in glob.iglob("car_damage/*"):
        base64_image = encode_image(image_path)
        image_url = f"data:image/jpeg;base64,{base64_image}"

        ground_truth = image_path.split("/")[-1][:-11].split("--")
        ground_truth = {
            "damaged_part": ground_truth[0],
            "damage_type": ground_truth[1],
            "damage_severity": ground_truth[2]
        }

        # image_url = "https://images.pickles.com.au/image/upload/f_auto,q_auto,w_2000/c_scale,g_south_east,l_pickles-logo-white_mipoap.png,o_60,w_0.4,fl_relative,x_0.02,y_0.02,fl_region_relative/v1726789702/asostfbejshw7ekod5xa.jpg"

        example = {
            "messages": [
                {
                    "role": "system",
                    "content": "You are an assistant that identifies which part of the car is damaged, damage type and the severity level of the damage"
                },
                {
                    "role": "user",
                    "content": "Identify which part of the car is damaged and the severity level of the damage. Possible values for the damaged parts are: [bumper, door, glass]. Possible values for the damage types are: [dent, scratch, shatter]. Possible values for the severity level are: [minor, major]."
                },
                {
                    "role": "user",
                    "content": [
                        {
                            "type": "image_url",
                            "image_url": {
                                "url": image_url
                            }
                        }
                    ]
                 },
                {
                    "role": "assistant", 
                    "content": f"{ground_truth}"
                }
            ]
        }

        example_str = json.dumps(example)
        fp.write(example_str + "\n")

In [ ]:
!head -n 1 car_damage_train.jsonl

In [ ]:
!tail -n 1 car_damage_train.jsonl

In [ ]:
response = client.files.create(
    file=open("car_damage_train.jsonl", "rb"),
    purpose="fine-tune"
)
print(response)

In [ ]:
file_id = response.id
print(file_id)

In [ ]:
response = client.fine_tuning.jobs.create(
    training_file=file_id,
    model="gpt-4o-2024-08-06",
    hyperparameters={"n_epochs": 1}
)

print("Fine-tune job is started")
ft_job_id = response.id

In [ ]:
response = client.fine_tuning.jobs.list_events(fine_tuning_job_id=ft_job_id, limit=10)
response

In [ ]:
response = client.fine_tuning.jobs.retrieve(ft_job_id)
print(response)

In [ ]:
result_files = response.result_files
result_files

In [ ]:
fine_tuned_model = response.fine_tuned_model
fine_tuned_model

In [89]:
image_url = "https://images.pickles.com.au/image/upload/f_auto,q_auto,w_2000/c_scale,g_south_east,l_pickles-logo-white_mipoap.png,o_60,w_0.4,fl_relative,x_0.02,y_0.02,fl_region_relative/v1726789702/asostfbejshw7ekod5xa.jpg"

In [ ]:
response = client.chat.completions.create(
    model=fine_tuned_model,
    messages=[
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": "Identify which part of the car is damaged and the severity level of the damage. Possible values for the damaged parts are: [bumper, door, glass]. Possible values for the damage types are: [dent, scratch, shatter]. Possible values for the severity level are: [minor, major]."
                },
                {
                    "type": "image_url",
                    "image_url": {
                        "url": image_url,
                    }
                }
            ]
        }
    ],
    max_tokens=100
)

print(response.choices[0])